# Interactive Map

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from shapely.geometry import Point, Polygon
import folium
from folium import Element
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
from base64 import b64encode
import glob
from rasterio.plot import show
import rasterio
from matplotlib.ticker import ScalarFormatter
import cmocean
import geopandas as gpd

## Processing all the data from total_df.ipynb 

In [2]:
# read in all casts processed in total_df.ipynb
total_df = pd.read_csv('data/total_df.csv')

# display the first 5 rows of the df 
total_df.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Pressure (dbar),Depth (m),Temperature (°C),Conductivity (µS/cm),Specific Conductance (µS/cm),Salinity (PSS),Sound Velocity (m/s),Density (kg/m³),Latitude,Longitude,UTC Time,File Name,Area,TestType,ImagePath
0,0,0,0,0,0.15,0.153151,9.153524,296.339874,433.834991,0.204679,1444.146359,999.934361,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211,MOR,cast,../data/profiles\profile_CC2435009_20250513_18...
1,1,1,1,1,0.45,0.458792,9.255136,295.835466,431.811837,0.203774,1444.561198,999.927172,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211,MOR,cast,../data/profiles\profile_CC2435009_20250513_18...
2,2,2,2,2,0.75,0.764653,9.132318,295.647002,433.089545,0.204300,1444.069489,999.938574,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211,MOR,cast,../data/profiles\profile_CC2435009_20250513_18...
3,3,3,3,3,1.05,1.070511,9.033097,296.121842,435.049797,0.205173,1443.672646,999.948268,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211,MOR,cast,../data/profiles\profile_CC2435009_20250513_18...
4,4,4,4,4,1.35,1.376366,8.925369,296.570547,437.092585,0.206079,1443.240202,999.958485,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211,MOR,cast,../data/profiles\profile_CC2435009_20250513_18...


In [3]:
# Extract the first row for each cast (based on File Name)
cast_locations = total_df.groupby("File Name").first().reset_index()

# Optional: if you want only lat/lon and filename
cast_locations = cast_locations[["Latitude", "Longitude", "File Name"]]

## Extracting the profiles from profiles.ipynb

In [4]:
# Create DataFrame
image_df = pd.read_csv('data/profile_data.csv')
image_df.head()

,Unnamed: 0,File Name,Latitude,Longitude,ImagePath,TestType,Temperature (°C),Conductivity (µS/cm),Salinity (PSS),Density (kg/m³)
0,0,CC2435009_20250513_181211,40.274148,-106.839900,data/profiles\profile_CC2435009_20250513_18121...,cast,9.153524,296.339874,0.204679,999.934361
1,1,CC2435009_20250513_181617,40.276507,-106.841752,data/profiles\profile_CC2435009_20250513_18161...,cast,9.902309,302.497925,0.204992,999.873001
2,2,CC2435009_20250513_182530,40.279905,-106.851733,data/profiles\profile_CC2435009_20250513_18253...,cast,12.425925,324.117126,0.206352,999.611949
3,3,CC2435009_20250513_182851,40.279532,-106.852019,data/profiles\profile_CC2435009_20250513_18285...,cast,12.494535,324.762451,0.206421,999.603737
4,4,CC2435009_20250513_194404,40.283628,-106.853450,data/profiles\profile_CC2435009_20250513_19440...,cast,12.716255,327.775024,0.207239,999.577265


In [5]:
total_df = total_df.drop('TestType_x', axis=1)
total_df = total_df.drop('ImagePath_y', axis=1)
total_df = total_df.drop('TestType_y', axis=1)
total_df = total_df.drop('ImagePath_x', axis=1)


KeyError: "['TestType_x'] not found in axis"

In [11]:
# Merge 'TestType' from cast_locations into total_df based on 'File Name'
cast_locations['TestType'] = image_df['TestType']
cast_locations['ImagePath'] = image_df['ImagePath']
total_df = total_df.merge(image_df[['File Name', 'TestType']], on='File Name', how='left')    
total_df = total_df.merge(image_df[['File Name', 'ImagePath']], on='File Name', how='left') 
total_df.head()
#total_df.to_csv('../data/total_df.csv')

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Pressure (dbar),Depth (m),Temperature (°C),Conductivity (µS/cm),Specific Conductance (µS/cm),Salinity (PSS),Sound Velocity (m/s),Density (kg/m³),Latitude,Longitude,UTC Time,File Name,Area,TestType,ImagePath
0,0,0,0,0,0.15,0.153151,9.153524,296.339874,433.834991,0.204679,1444.146359,999.934361,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211,MOR,cast,data/profiles\profile_CC2435009_20250513_18121...
1,1,1,1,1,0.45,0.458792,9.255136,295.835466,431.811837,0.203774,1444.561198,999.927172,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211,MOR,cast,data/profiles\profile_CC2435009_20250513_18121...
2,2,2,2,2,0.75,0.764653,9.132318,295.647002,433.089545,0.204300,1444.069489,999.938574,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211,MOR,cast,data/profiles\profile_CC2435009_20250513_18121...
3,3,3,3,3,1.05,1.070511,9.033097,296.121842,435.049797,0.205173,1443.672646,999.948268,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211,MOR,cast,data/profiles\profile_CC2435009_20250513_18121...
4,4,4,4,4,1.35,1.376366,8.925369,296.570547,437.092585,0.206079,1443.240202,999.958485,40.274148,-106.8399,2025-05-13 18:12:11,CC2435009_20250513_181211,MOR,cast,data/profiles\profile_CC2435009_20250513_18121...


## Setting up the interactive map using Folium
- Make polygons to categorize the region each measurement was taken in

In [12]:
# ---- Define Polygons ----
polygon_coords = {
    'DOC': [(40.28429, -106.85365), (40.28360, -106.85341), (40.28240, -106.85488), (40.28356, -106.85524)],
    'HAR': [(40.28424, -106.85348), (40.28565, -106.85185), (40.28797, -106.85680), (40.28793, -106.85836)],
    'MOR': [(40.28003, -106.84560), (40.28021, -106.84274), (40.27252, -106.83760), (40.27183, -106.84056)],
    'GLZ': [(40.27291, -106.84515), (40.27610, -106.85043), (40.27509, -106.85397), (40.27129, -106.84970)],
    'KEY': [(40.27705, -106.86361), (40.27700, -106.86697), (40.28124, -106.86839), (40.28180, -106.86597)],
    'LIL': [(40.27265, -106.86163), (40.27315, -106.85961), (40.27114, -106.85906), (40.27115, -106.86068)],
    'ASP': [(40.272286, -106.876870),(40.272286, -106.876599),(40.272016, -106.876599),(40.272016, -106.876870)]
}
polygons = {name: Polygon(coords) for name, coords in polygon_coords.items()}

# ---- Classify Points by Area ----
def get_area(lat, lon):
    point = Point(lat, lon)
    for name, poly in polygons.items():
        if poly.contains(point):
            return name
    return 'MID'

# ---- Process Data ----
cast_locations['Area'] = cast_locations.apply(lambda row: get_area(row['Latitude'], row['Longitude']), axis=1)
total_df['Area'] = total_df.apply(lambda row: get_area(row['Latitude'], row['Longitude']), axis=1)
cast_locations['Date'] = pd.to_datetime(cast_locations['File Name'].str.extract(r'_(\d{8})_')[0], format='%Y%m%d')

In [13]:
# ---- Color Map for Areas ----
area_colors = {
    'DOC': 'red',
    'HAR': 'cyan',
    'MOR': 'purple',
    'GLZ': 'orange',
    'KEY': 'darkblue',
    'LIL': 'darkred',
    'ASP': 'pink',
    'MID': 'gray'
}

# ---- Legend HTML ----
legend_html = """
<div style="
    position: fixed; 
    bottom: 30px; left: 30px; width: 150px; z-index:500;
    background-color: white; padding: 10px; border: 2px solid grey;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3); font-size:14px;">
<b>Areas</b><br>
<i style='background:red; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Dock<br>
<i style='background:cyan; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Harding Cove<br>
<i style='background:purple; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Morrison Cove<br>
<i style='background:orange; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Glizzy Cove<br>
<i style='background:darkblue; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Keystone Cove<br>
<i style='background:darkred; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Small Inlet<br>
<i style='background:pink; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Aspen View<br>
<i style='background:gray; width:10px; height:10px; float:left; margin-right:8px; display:inline-block'></i>Middle of Reservoir
</div>
"""

# ---- Widget Setup ----
date_picker = widgets.DatePicker(description='Pick a date', value=cast_locations['Date'].min())
toggle_button = widgets.ToggleButton(value=True, description="Show All Data", button_style='info')
map_output = widgets.Output()

def update_map(change=None):
    with map_output:
        clear_output(wait=True)
        if toggle_button.value:
            filtered = cast_locations
        else:
            selected_date = date_picker.value
            filtered = cast_locations[cast_locations['Date'] == pd.to_datetime(selected_date)]
        
        # Create map
        m = folium.Map(
            location=[filtered["Latitude"].mean(), filtered["Longitude"].mean()],
            zoom_start=13
        )

        # ---- Plot Polygons ----
#        for name, coords in polygon_coords.items():
#            folium.Polygon(
#                locations=coords,
#                color=area_colors[name],
#                fill=True,
#                fill_opacity=0.15,
#                popup=f"{name} Area"
#            ).add_to(m)
        
        # Plot points
        for _, row in filtered.iterrows():

            if row['TestType'] == 'point':
                # Show raw measurement values
                point = total_df[total_df['File Name'] == row['File Name']]
                popup_html = f"""
                <b>File:</b> {row['File Name']}<br>
                <b>Test Type:</b> {'Point'}<br>
                <b>Lat:</b> {row['Latitude']}<br>
                <b>Lon:</b> {row['Longitude']}<br>
                <b>Area:</b> {row['Area']}<br>
                <b>Temperature (°C):</b> {point['Temperature (°C)'].iloc[0]}<br>
                <b>Conductivity (µS/cm):</b> {point['Conductivity (µS/cm)'].iloc[0]}<br>
                <b>Salinity (PSS):</b> {point['Salinity (PSS)'].iloc[0]}<br>
                <b>Density (kg/m³):</b> {point['Density (kg/m³)'].iloc[0]}<br>
                """
                iframe = folium.IFrame(popup_html, width=300, height=250)
                popup = folium.Popup(iframe, max_width=300)   

            elif row['TestType'] == 'cast':
                with open(row['ImagePath'], 'rb') as f:
                    encoded = b64encode(f.read()).decode()
                popup_html = f"""
                    <div>
                        <a href="data:image/png;base64,{encoded}" target="_blank">
                            <img src="data:image/png;base64,{encoded}" width="700">
                        </a><br>
                        <b>Date:</b> {row['Date'].date()}<br>
                        <b>Test Type:</b> {'Cast'}<br>
                        <b>File:</b> {row['File Name']}<br>
                        <b>Lat:</b> {row['Latitude']}<br>
                        <b>Lon:</b> {row['Longitude']}<br>
                        <b>Area:</b> {row['Area']}
                    </div>
                """
                iframe = folium.IFrame(html=popup_html, width=450, height=300)
                popup = folium.Popup(iframe, max_width=450)    
            
            folium.CircleMarker(
                location=[row["Latitude"], row["Longitude"]],
                radius=5,
                popup = popup,
                color=area_colors.get(row["Area"], "gray"),
                fill=True,
                fill_color=area_colors.get(row["Area"], "gray"),
                fill_opacity=0.9
            ).add_to(m)

        m.get_root().html.add_child(Element(legend_html))
        display(m)
        m.save("map.html")

# ---- Connect Handlers ----
date_picker.observe(update_map, names='value')
toggle_button.observe(update_map, names='value')

# ---- Show UI ----
display(widgets.HBox([date_picker, toggle_button]))
display(map_output)
update_map()  # Initial call




Output()